In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import xarray as xr

from google.colab import drive
drive.mount('/content/gdrive')

import features

In [ ]:
cmorph = np.float32(np.load('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/5d_feature_tensors/cmorph_feature_tensor.npy'))
imerg = np.float32(np.load('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/5d_feature_tensors/imerg_feature_tensor.npy'))
tamsat = np.float32(np.load('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/5d_feature_tensors/tamsat_feature_tensor.npy'))
chirps = np.float32(np.load('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/5d_feature_tensors/chirps_feature_tensor.npy'))

cmorph_val = cmorph[1461:]
imerg_val = imerg[1461:]
tamsat_val = tamsat[1461:]
chirps_val = chirps[1461:]

#features_train = np.stack((cmorph_train[:,:,:,:,0], imerg_train[:,:,:,:,0], tamsat_train[:,:,:,:,0], chirps_train[:,:,:,:,0]), axis = 4)
features_val = np.stack((cmorph_val[:,:,:,:,0], imerg_val[:,:,:,:,0], tamsat_val[:,:,:,:,0], chirps_val[:,:,:,:,0]), axis = 4)
#print(features_train.shape)
print(features_val.shape)

(363, 4, 285, 245, 4)


In [ ]:
model1 = tf.keras.models.load_model('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/training_convlstm_history/model_stacked_1.h5')
model2 = tf.keras.models.load_model('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/training_convlstm_history/model_second.h5')

In [ ]:
prediction1 = model1.predict(features_val, batch_size = 32)
np.save('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/training_convlstm_history/model1_predictions', prediction1)
prediction2 = model2.predict(features_val, batch_size = 32)
np.save('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/training_convlstm_history/model2_predictions', prediction2)

clear


In [ ]:
example_netcdf = xr.open_dataset('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/TAMSAT_regridded.nc')
dataset_netcdf = example_netcdf.copy()

In [ ]:
dataset_netcdf = dataset_netcdf.loc[dict(time=slice("2020-01-04", "2020-12-31"))]
dataset_netcdf = dataset_netcdf.drop_vars(('rfe','rfe_filled'))

In [ ]:
dataset_netcdf['predict_precip_1'] = (['time', 'lat', 'lon'], prediction1[:,:,:,0])
dataset_netcdf['predict_precip_2'] = (['time', 'lat', 'lon'], prediction2[:,:,:,0])

In [ ]:
dataset_netcdf.to_netcdf('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/convlstm_prediction.nc')

In [ ]:
model2.summary()

Model: "model_stacked_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_4 (ConvLSTM2D)  (None, 4, 285, 245, 20)   17360     
_________________________________________________________________
batch_normalization_5 (Batch (None, 4, 285, 245, 20)   80        
_________________________________________________________________
conv_lst_m2d_5 (ConvLSTM2D)  (None, 285, 245, 20)      28880     
_________________________________________________________________
batch_normalization_6 (Batch (None, 285, 245, 20)      80        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 285, 245, 1)       21        
Total params: 46,421
Trainable params: 46,341
Non-trainable params: 80
_________________________________________________________________
